In [76]:

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


driver = webdriver.Chrome()
print("Navigateur ouvert.")


url = "https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats"
driver.get(url)
print(f"Connexion à : {url}")

time.sleep(3) 


Navigateur ouvert.
Connexion à : https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats


In [78]:
teams_table= driver.find_element(By.CSS_SELECTOR,'.stats_table')

print(teams_table.text)
# driver.quit()

Rk Squad MP W D L GF GA GD Pts Pts/MP xG xGA xGD xGD/90 Attendance Top Team Scorer Goalkeeper Notes
1 Liverpool 38 25 9 4 86 41 +45 84 2.21 82.2 38.6 +43.6 +1.15 60,324 Mohamed Salah - 29 Alisson → Champions League via league finish
2 Arsenal 38 20 14 4 69 34 +35 74 1.95 59.9 34.4 +25.5 +0.67 60,251 Kai Havertz - 9 David Raya → Champions League via league finish
3 Manchester City 38 21 8 9 72 44 +28 71 1.87 68.1 47.7 +20.4 +0.54 52,756 Erling Haaland - 22 Ederson → Champions League via league finish
4 Chelsea 38 20 9 9 64 43 +21 69 1.82 67.8 47.3 +20.5 +0.54 39,672 Cole Palmer - 15 Robert Sánchez → Champions League via league finish
5 Newcastle Utd 38 20 6 12 68 47 +21 66 1.74 63.8 45.5 +18.3 +0.48 52,187 Alexander Isak - 23 Nick Pope → Champions League via league finish
6 Aston Villa 38 19 9 10 58 51 +7 66 1.74 56.1 50.1 +5.9 +0.16 42,079 Ollie Watkins - 16 Emiliano Martínez → Europa League via league finish
7 Nott'ham Forest 38 19 8 11 58 46 +12 65 1.71 45.5 48.9 -3.4 -0.09 30,059 Ch

In [79]:
import os
teams_name=teams_table.find_elements(By.CSS_SELECTOR,'tbody > tr td[data-stat="team"] a')
base_path = "../../data/raw/bronze/teams/"  
os.makedirs(base_path, exist_ok=True)

print(f"{len(teams_name)} équipes trouvées :")
dictionary_teams = {}
for team in teams_name:
    team_name = team.text.strip() 
    team_link = team.get_attribute("href")  
    dictionary_teams[team_name]=team_link
    print(team_name)
    team_folder=os.path.join(base_path,team_name)
    os.makedirs(team_folder, exist_ok=True)
    
print(dictionary_teams.items())
print(" Tous les dossiers ont été créés avec succès.")

20 équipes trouvées :
Liverpool
Arsenal
Manchester City
Chelsea
Newcastle Utd
Aston Villa
Nott'ham Forest
Brighton
Bournemouth
Brentford
Fulham
Crystal Palace
Everton
West Ham
Manchester Utd
Wolves
Tottenham
Leicester City
Ipswich Town
Southampton
dict_items([('Liverpool', 'https://fbref.com/en/squads/822bd0ba/2024-2025/Liverpool-Stats'), ('Arsenal', 'https://fbref.com/en/squads/18bb7c10/2024-2025/Arsenal-Stats'), ('Manchester City', 'https://fbref.com/en/squads/b8fd03ef/2024-2025/Manchester-City-Stats'), ('Chelsea', 'https://fbref.com/en/squads/cff3d9bb/2024-2025/Chelsea-Stats'), ('Newcastle Utd', 'https://fbref.com/en/squads/b2b47a98/2024-2025/Newcastle-United-Stats'), ('Aston Villa', 'https://fbref.com/en/squads/8602292d/2024-2025/Aston-Villa-Stats'), ("Nott'ham Forest", 'https://fbref.com/en/squads/e4a775cb/2024-2025/Nottingham-Forest-Stats'), ('Brighton', 'https://fbref.com/en/squads/d07537b9/2024-2025/Brighton-and-Hove-Albion-Stats'), ('Bournemouth', 'https://fbref.com/en/squads/

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd 
for team,link in dictionary_teams.items():
    driver.get(link)
    time.sleep(4)

    soup=BeautifulSoup(driver.page_source,'html.parser')
    ## Extraction  des joueurs
    table=soup.find('table',{'id':'stats_standard_9'})


    thead = table.find("thead")
    header_rows = thead.find_all("tr")
    
    header_rows[0].decompose()
    
    df=pd.read_html(str(table))[0]
    colonnes_interessees = ["Player", "Nation", "Pos", "Age", "MP","Starts","Min","90s","Gls", "Ast", "G+A", "G-PK","PK","PKatt","CrdY","CrdR"]
    df_players = df[colonnes_interessees]
    file_path=os.path.join(base_path,team,f"{team}_players_stats.csv")
    df_players.to_csv(file_path,index=False)
    print(f"Fichier enregistré pour l'équipe {team} : {file_path}")

## Extraction  d'équipe
    table2=soup.find('table',{'id':"matchlogs_for"})
    colonnes_matchs_interessees=["Date","Comp","Round","Day","Venue","Result","GF","GA","Opponent","xG","xGA","Poss","Attendance","Captain","Formation","Opp Formation","Referee"]
    df_matchs=pd.read_html(str(table2))[0]
    df_matchs_equipes=df_matchs[colonnes_matchs_interessees]
    file_path_equipes=os.path.join(base_path,team,f"{team}_matchs_stats.csv")
    df_matchs_equipes.to_csv(file_path_equipes,index=False)


C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(table))[0]
C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_matchs=pd.read_html(str(table2))[0]


Fichier enregistré pour l'équipe Liverpool : ../../data/raw/bronze/teams/Liverpool\Liverpool_players_stats.csv


C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(table))[0]
C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_matchs=pd.read_html(str(table2))[0]


Fichier enregistré pour l'équipe Arsenal : ../../data/raw/bronze/teams/Arsenal\Arsenal_players_stats.csv


C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(table))[0]
C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_matchs=pd.read_html(str(table2))[0]


...

C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(table))[0]
C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_matchs=pd.read_html(str(table2))[0]


Fichier enregistré pour l'équipe Leicester City : ../../data/raw/bronze/teams/Leicester City\Leicester City_players_stats.csv


C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(table))[0]
C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_matchs=pd.read_html(str(table2))[0]


Fichier enregistré pour l'équipe Ipswich Town : ../../data/raw/bronze/teams/Ipswich Town\Ipswich Town_players_stats.csv
Fichier enregistré pour l'équipe Southampton : ../../data/raw/bronze/teams/Southampton\Southampton_players_stats.csv


C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(table))[0]
C:\Users\oussa\AppData\Local\Temp\ipykernel_38952\3071462437.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_matchs=pd.read_html(str(table2))[0]


In [ ]:
import pandas as pd

df = pd.read_csv("../../data/raw/bronze/teams/Arsenal/Arsenal_players_stats.csv")

mask = df.apply(lambda row: row.astype(str).str.contains("Playing Time|Performance").any(), axis=1)
df = df[~mask]

df = df[~df["Player"].isin(["Player", "Ast"])]

df = df.reset_index(drop=True)

print(df.head(20))
print(df.shape)


                Player   Nation    Pos Age  MP Starts   Min   90s Gls Ast G+A  \
0           David Raya   es ESP     GK  28  38     38  3420  38.0   0   0   0   
1       William Saliba   fr FRA     DF  23  35     35  3039  33.8   2   0   2   
2          Declan Rice  eng ENG     MF  25  35     33  2825  31.4   4   7  11   
3        Thomas Partey   gh GHA  MF,DF  31  35     31  2797  31.1   4   2   6   
4     Leandro Trossard   be BEL     FW  29  38     28  2546  28.3   8   7  15   
5    Gabriel Magalhães   br BRA     DF  26  28     28  2363  26.3   3   1   4   
6       Jurriën Timber   nl NED     DF  23  30     27  2417  26.9   1   3   4   
7      Martin Ødegaard   no NOR     MF  25  30     26  2325  25.8   3   8  11   
8   Gabriel Martinelli   br BRA  FW,MF  23  33     25  2290  25.4   8   4  12   
9          Kai Havertz   de GER  FW,MF  25  23     21  1875  20.8   9   3  12   
10         Bukayo Saka  eng ENG  FW,MF  22  25     20  1729  19.2   6  10  16   
11        Mikel Merino   es 